In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
pip install neurokit2

     |████████████████████████████████| 993kB 5.0MB/s 


In [3]:
pip install biosppy==0.6.1

     |████████████████████████████████| 81kB 4.1MB/s 


In [4]:
pip install mne

     |████████████████████████████████| 7.0MB 4.7MB/s 


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time 
import re
import csv
import scipy.io
import biosppy
import mne
import neurokit2 as nk
import ast
import os
import scipy.io
from sklearn.preprocessing import LabelEncoder
import time
import datetime
from datetime import datetime
import glob
from scipy.stats import zscore, norm
from neurokit2 import eda_phasic
from scipy.stats import linregress
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('/content/gdrive/My Drive/Food_VR/GSR_Preprocessing/W&S_D_Shimmer_GSR/')

from ******* import TimeStamp_Conversion, column_formatting, Annotation_timestamp, Shimmers_csv2DF, Sliding_Window_GSRFeatureExt 

In [6]:
# this function is to convert the TimeStamp column (first column) from Unix Epoch time to standard datetime format
def TimeStamp_Conversion(ts):
  """
  we have a unix epoch time in milliseconds i.e, a string with a length of 13 charcters example:1.5789360034388428E12
  
  parameters:
  -----
  ts = Epoch timesatmp in milliseconds.

  Returns:
  -----
  Std_Unix = standard epoch timestamp in seconds.

  """

  float_Input = float(ts) # converting the string-type(1.5789360034388428E12) Unix Epoch to float-type(1578936003.4388428).

  # float input is divided by 1000 to convert the Unix epoch in milliseconds to seconds 
  Std_Unix = float_Input/1000

  datetime_Input = datetime.fromtimestamp(Std_Unix) 
  # datetime.fromtimestamp converts the unix epoch in seconds to datetime returns example:datetime.datetime(2020, 1, 13, 17, 20, 3, 438843)

  return Std_Unix

In [7]:
def column_formatting(Timestamp_DF):
  """
  Column names of Timestamp annotation excel have column index attached to column name as we only need column name we are parsing column names.

  Parameters:
  -----
  Timestamp_DF = Input the DF after reading the timestamp annotationexcel file to  get list of column names['A1- ECG baseline start','B1- ECG baseline end',.....].

  Returns:
  -----
  Parsed_ColumnNames = list of parsed column names. ['ECG baseline start','ECG baseline end',....]

  """

  Parsed_colnames = ['Subject_ID'] ## Column with Participant ID is not named, so declaring first column as Subject_ID to an empty list

  for index in range(1,len(Timestamp_DF.columns)): ## Looping through the list of timestamp annotation columns list
    column = Timestamp_DF.columns[index][4:].lstrip() ## Drop first 3 indices of each column and strip space(" ") if present as left most
    Parsed_colnames.append(column) ## appending each column name after parsing

  return Parsed_colnames ## returns list fo parsed col names

In [8]:
def Annotation_timestamp(timestamp_path, sheet_name):
  """
  This function is to change the column names of timestamp annotations table and convert timestamps from milliseconds to standart epoch format of seconds.

  Parameters:
  -----
  timestamp_path = path to the directory of file location
  sheet_name =  there are two sheets present in the file, we work on file named D.

  Results:
  -----
  VR_Timestamps_D = Clean dataframe of timestamp annotations table.

  """

  VR_TimeStamps_D = pd.read_excel(Timestamp_path, sheet_name) ## read timestamp annotation file
  Parsed_colnames = column_formatting(VR_TimeStamps_D) ## using the column_formatting function defined earlier parse columns
  VR_TimeStamps_D.columns = Parsed_colnames ## Change colnames of Dataframe using the parsed list of col names
  
  ## As timestamp is in string format and in milli seconds iterating through each column to change the timestamp to standard epoch format.
  for col in VR_TimeStamps_D.columns: 
    ## Using Timestamp_Conversion function and lambda fucntion to map the function to each row of the column.
    if col == 'Subject_ID':
      pass
    else:
      VR_TimeStamps_D[col] = VR_TimeStamps_D[col].map(lambda instance: TimeStamp_Conversion(instance)) 

  return VR_TimeStamps_D

In [9]:
def Shimmers_csv2DF(path,filename):
  """
  This function is to read Shimmer data files and create a dataframe from tidy shimmers csv tables.

  Parameters:
  -----
  path = path to directory of shimmers file folder.

  filename = name of the file to be loaded.

  Results:
  -----
  Dataframe = organized and structured Shimmers Data.

  """

  with open(path + '/' + filename, 'r',) as file: # read the file
    reader = csv.reader(file)

    lists_eachrow = []
    for row in reader:
      lists_eachrow.append(row) # append each row in reader to a list

  del lists_eachrow[0] # del first row of list as it is only about \t delimiter used

  newlists = [] 
  # loop through the list of lists and split columnar values using the delimiter 
  for list_row in lists_eachrow:
    for row in list_row:
      newlists.append(list(row.split('\t')))
  # Extract subjectID from the file name for future use
  filename_parse = filename.replace("_", " ")
  Participant_ID = ast.literal_eval(re.findall(r'\b\d+\b', filename_parse)[0])
  
  # create dataframe from the list of columnar values 
  Dataframe = pd.DataFrame(newlists, columns = newlists[0])
  Dataframe = Dataframe.drop([0,1]) # drop columns 1 and 2 as are column names and units which we already have for new dataframe.
  Dataframe.reset_index(drop=True, inplace=True) # reset index

  return Dataframe, Participant_ID 

In [10]:
def Sliding_Window_GSRFeatureExt(GSR_16Hz,Participant_ID, start_window, overlap, window_length):
  """
  This function is to create a dataframe consisting of statistical features extracted using sliding window technique on Phasic and Tonic components of GSR signal.

  Parameters:
  -----
  GSR_16Hz = This the Dataframe consisting of Phasic and Tonic components of GSR signal along with epoch timestamp of each instance.

  Participant_ID = Subject ID extracted in Shimmers_csv2DF function from the file name.
  
  start_window = 0, starting index of the sliding window
  
  overlap = 50, sliding window tech. with 50 overlap
  
  window_length = 100, length of each window (100 rows at once)

  Results:
  -----
  Dataframe consisting of statistical features like Mean Phasic&Tonic components, std Phasic&Tonic components, Count and Mean of Phasic&Tonic Peaks, 
  Min and Max, Slope, AUC of Phasic&Tonic components for each window of length 100 and overlap of 50.

 """
  # creating an array of phasic, tonic and timestamps from the GSR_16Hz dataframe, will be easy to calculate statistical features of numpy array.
  phasic  = np.array(GSR_16Hz['phasic']) 
  tonic  = np.array(GSR_16Hz['tonic'])
  timestamp = np.array(GSR_16Hz['TimeStamp'])
  dummy = list(range(0,len(GSR_16Hz))) # this is to use as secondary axis to calculate slope.

  end_window = window_length
  Start_timestamp = []
  Mean_timestamp = []
  end_timestamp = []
  Mean_phasic = []
  Mean_tonic = []
  Std_phasic = []
  Std_tonic = []
  CountPeak_phasic = []
  CountPeak_tonic = []
  MeanPeak_phasic = []
  MeanPeak_tonic = []
  Min_phasic = []
  Max_phasic = []
  Min_tonic = []
  Max_tonic = []
  Slope_tonic = []
  Slope_phasic = []
  AUC_tonic = []
  AUC_phasic = []
  while abs(end_window-window_length) <= len(GSR_16Hz): # loop conditon to carry out windowing and calculate features till the end 

    # creating start, mean and end timestamps helps to  make sure that each window completely falls in annotation period
    Start_timestamp.append(timestamp[start_window]) 
    Mean_timestamp.append(np.mean(timestamp[start_window:end_window]))
    try:
      end_timestamp.append(timestamp[end_window])
    except IndexError: 
      # index of last window might exceed the last timestamp of our data, if this happens it will use last timestamp of data as end timsatmp of last window
      end_timestamp.append(timestamp[-1])

    # using numpy to compute statistucal features like mean, std, count
    Mean_phasic.append(np.mean(phasic[start_window:end_window]))
    Mean_tonic.append(np.mean(tonic[start_window:end_window]))
    Std_phasic.append(np.std(phasic[start_window:end_window]))
    Std_tonic.append(np.std(tonic[start_window:end_window]))
    # scipy.signal.find_peaks exctracts the peaks present in teh signal
    CountPeak_phasic.append(scipy.signal.find_peaks(phasic[start_window:end_window])[0].size)
    CountPeak_tonic.append(scipy.signal.find_peaks(tonic[start_window:end_window])[0].size)
    MeanPeak_phasic.append(np.mean(scipy.signal.find_peaks(phasic[start_window:end_window])[0]))
    MeanPeak_tonic.append(np.mean(scipy.signal.find_peaks(tonic[start_window:end_window])[0]))
    Min_phasic.append(min(phasic[start_window:end_window]))
    Max_phasic.append(max(phasic[start_window:end_window]))
    Min_tonic.append(min(tonic[start_window:end_window]))
    Max_tonic.append(max(tonic[start_window:end_window]))
    # slope is computed using linregress module
    Slope_tonic.append(linregress(dummy[start_window:end_window],tonic[start_window:end_window])[0])
    Slope_phasic.append(linregress(dummy[start_window:end_window],phasic[start_window:end_window])[0])
    # Area under curve is calculated using sk learn metrics
    AUC_tonic.append(metrics.auc(dummy[start_window:end_window],tonic[start_window:end_window]))
    AUC_phasic.append(metrics.auc(dummy[start_window:end_window],phasic[start_window:end_window]))
      
    # to increment start and end for next window
    start_window += overlap
    end_window += overlap
  
  # create Data frame from the lists of values appended to each feature.
  Feature_DF = pd.DataFrame()
  Feature_DF['Start_timestamp'] = Start_timestamp
  Feature_DF['Mean_timestamp'] = Mean_timestamp
  Feature_DF['End_timestamp'] = end_timestamp
  Feature_DF['Mean_phasic'] = Mean_phasic
  Feature_DF['Mean_tonic'] = Mean_tonic
  Feature_DF['Std_phasic'] = Std_phasic
  Feature_DF['Std_tonic'] = Std_tonic
  Feature_DF['CountPeak_phasic'] = CountPeak_phasic
  Feature_DF['CountPeak_tonic'] = CountPeak_tonic
  Feature_DF['MeanPeak_phasic'] = MeanPeak_phasic
  Feature_DF['MeanPeak_tonic'] = MeanPeak_tonic
  Feature_DF['Min_phasic'] = Min_phasic
  Feature_DF['Max_phasic'] = Max_phasic
  Feature_DF['Min_tonic'] = Min_tonic
  Feature_DF['Max_tonic'] = Max_tonic
  Feature_DF['Slope_tonic'] = Slope_tonic
  Feature_DF['Slope_phasic'] = Slope_phasic
  Feature_DF['AUC_tonic'] = AUC_tonic
  Feature_DF['AUC_phasic'] = AUC_phasic
  Feature_DF['Subject_ID'] = Participant_ID

  return Feature_DF

In [11]:
def GSR_features(path,Timestamp_path,method = 'cvxEDA',sheet_name = 'D',start_window=0, overlap=50, window_length=100):

  os.chdir(path)
  list_csv = glob.glob('*.{}'.format('txt'))
  list_csv.extend(glob.glob('*.{}'.format('csv')))

  Data = pd.DataFrame()
  for index in list_csv:
    Dataframe, Participant_ID = Shimmers_csv2DF(path,index)

    GSR_128Hz = pd.DataFrame(Dataframe, columns = ['Shimmer_89C4_Timestamp_Unix_CAL','Shimmer_89C4_GSR_Skin_Conductance_CAL'])
    GSR_128Hz['Timestamp_s'] = GSR_128Hz['Shimmer_89C4_Timestamp_Unix_CAL'].map(lambda instance: TimeStamp_Conversion(instance))

    signal = list(GSR_128Hz['Shimmer_89C4_GSR_Skin_Conductance_CAL'])
    signal = list(map(lambda index : ast.literal_eval(index),signal))

    signal = mne.filter.resample(signal, down = 8)
    len_downsample = len(signal)
    start_time = list(GSR_128Hz['Timestamp_s'])[0]
    end_time = list(GSR_128Hz['Timestamp_s'])[-1]
    duration= end_time-start_time
    downsample_freq = len_downsample/duration


    list_ts = np.array(np.linspace(start_time,end_time,len_downsample))

    GSR_16Hz = pd.DataFrame()
    GSR_16Hz['TimeStamp'] = list_ts
    GSR_16Hz['GSRC'] = signal

    GSR_16Hz['GSRC'] = GSR_16Hz['GSRC'].ewm(span = 16).mean()
    min_max_scaler = MinMaxScaler()
    GSR_16Hz['GSRC'] = min_max_scaler.fit_transform(np.array(GSR_16Hz['GSRC']).reshape(-1, 1))

    eda = (GSR_16Hz['GSRC'])
    Phasic_Tonic_DF = eda_phasic(eda,downsample_freq, method)

    GSR_16Hz['phasic'] = Phasic_Tonic_DF['EDA_Phasic']
    GSR_16Hz['tonic'] = Phasic_Tonic_DF['EDA_Tonic']

    Feature_DF = Sliding_Window_GSRFeatureExt(GSR_16Hz,Participant_ID, start_window, overlap, window_length)

    VR_TimeStamps_D = Annotation_timestamp(Timestamp_path, sheet_name)

    VRBaseline_start = VR_TimeStamps_D.loc[VR_TimeStamps_D['Subject_ID'] == Participant_ID,'VR baseline start'].iloc[0]
    VRBaseline_end = VR_TimeStamps_D.loc[VR_TimeStamps_D['Subject_ID'] == Participant_ID,'VR baseline end'].iloc[0]
    Speech_start = VR_TimeStamps_D.loc[VR_TimeStamps_D['Subject_ID'] == Participant_ID,'Emotion-induction speech start'].iloc[0]
    Speech_end = VR_TimeStamps_D.loc[VR_TimeStamps_D['Subject_ID'] == Participant_ID,'Emotion-induction speech end'].iloc[0]
    Food_start = VR_TimeStamps_D.loc[VR_TimeStamps_D['Subject_ID'] == Participant_ID,'Food selection start'].iloc[0]
    Food_end = VR_TimeStamps_D.loc[VR_TimeStamps_D['Subject_ID'] == Participant_ID,'food selection end'].iloc[0]

    Feature_DF['Event'] = ''

    for i in range(0,len(Feature_DF)):
      if VRBaseline_start <= Feature_DF['Start_timestamp'][i] <= VRBaseline_end and VRBaseline_start <= Feature_DF['Mean_timestamp'][i] <= VRBaseline_end and VRBaseline_start <= Feature_DF['End_timestamp'][i] <= VRBaseline_end:
        Feature_DF['Event'][i] = 'VR Baseline'
      elif Speech_start <= Feature_DF['Start_timestamp'][i] <= Speech_end and Speech_start <= Feature_DF['Mean_timestamp'][i] <= Speech_end and Speech_start <= Feature_DF['End_timestamp'][i] <= Speech_end:
        Feature_DF['Event'][i] = 'Speech Emotion'
      elif Food_start <= Feature_DF['Start_timestamp'][i] <= Food_end and Food_start <= Feature_DF['Mean_timestamp'][i] <= Food_end and Food_start <= Feature_DF['End_timestamp'][i] <= Food_end:
        Feature_DF['Event'][i] = 'Food Selection'
      else:
        Feature_DF.drop(i, inplace = True)

    Data = Data.append(Feature_DF, ignore_index=True)
    
   # Data.to_csv(path+f'/Feature_data.csv')
  
  return Data

In [12]:
path = '/content/gdrive/My Drive/Food_VR/GSR_Preprocessing/W&S_D_Shimmer_GSR'
Timestamp_path = '/content/gdrive/My Drive/Food_VR/GSR_Preprocessing/W&S_D_Shimmer_GSR/VR Timestamps for Phase B & D_W&SP20.xlsx'
Data = GSR_features(path,Timestamp_path,method = 'cvxEDA',sheet_name = 'D',start_window=0, overlap=50, window_length=100)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be s

In [13]:
Data.head()

,Start_timestamp,Mean_timestamp,End_timestamp,Mean_phasic,Mean_tonic,Std_phasic,Std_tonic,CountPeak_phasic,CountPeak_tonic,MeanPeak_phasic,MeanPeak_tonic,Min_phasic,Max_phasic,Min_tonic,Max_tonic,Slope_tonic,Slope_phasic,AUC_tonic,AUC_phasic,Subject_ID,Event
0,1.579628e+09,1.579628e+09,1.579628e+09,0.137788,0.349071,0.025615,0.007046,1,0,53.0,NaN,0.076014,0.175401,0.342443,0.366755,-0.000206,-0.000422,34.549886,13.644040,877,VR Baseline
1,1.579628e+09,1.579628e+09,1.579628e+09,0.155515,0.353225,0.035974,0.011774,2,0,35.5,NaN,0.076014,0.197143,0.342443,0.381280,0.000375,0.000533,34.959472,15.382076,877,VR Baseline
2,1.579628e+09,1.579628e+09,1.579628e+09,0.157048,0.384361,0.029731,0.025013,1,0,18.0,NaN,0.098688,0.197143,0.348055,0.430875,0.000863,-0.000913,38.046656,15.581668,877,VR Baseline
3,1.579628e+09,1.579628e+09,1.579628e+09,0.097678,0.430603,0.039593,0.028115,0,0,NaN,NaN,0.037392,0.165216,0.382184,0.475994,0.000974,-0.001369,42.631168,9.666480,877,VR Baseline
4,1.579628e+09,1.579628e+09,1.579628e+09,0.041411,0.472420,0.025445,0.020160,0,0,NaN,NaN,0.008732,0.097300,0.431889,0.498678,0.000688,-0.000864,46.776745,4.088084,877,VR Baseline


In [14]:
DataFrame = Data.drop(['End_timestamp','Start_timestamp','Mean_timestamp' ], axis=1)

In [15]:
DataFrame.shape

(1462, 18)

In [16]:
DataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Mean_phasic       1462 non-null   float64
 1   Mean_tonic        1462 non-null   float64
 2   Std_phasic        1462 non-null   float64
 3   Std_tonic         1462 non-null   float64
 4   CountPeak_phasic  1462 non-null   int64  
 5   CountPeak_tonic   1462 non-null   int64  
 6   MeanPeak_phasic   1016 non-null   float64
 7   MeanPeak_tonic    281 non-null    float64
 8   Min_phasic        1462 non-null   float64
 9   Max_phasic        1462 non-null   float64
 10  Min_tonic         1462 non-null   float64
 11  Max_tonic         1462 non-null   float64
 12  Slope_tonic       1462 non-null   float64
 13  Slope_phasic      1462 non-null   float64
 14  AUC_tonic         1462 non-null   float64
 15  AUC_phasic        1462 non-null   float64
 16  Subject_ID        1462 non-null   int64  


In [17]:
DataFrame.isnull().sum()

Mean_phasic            0
Mean_tonic             0
Std_phasic             0
Std_tonic              0
CountPeak_phasic       0
CountPeak_tonic        0
MeanPeak_phasic      446
MeanPeak_tonic      1181
Min_phasic             0
Max_phasic             0
Min_tonic              0
Max_tonic              0
Slope_tonic            0
Slope_phasic           0
AUC_tonic              0
AUC_phasic             0
Subject_ID             0
Event                  0
dtype: int64

In [18]:
DataFrame = DataFrame.fillna(0)

In [19]:
DataFrame.Event.value_counts()

Food Selection    489
Speech Emotion    487
VR Baseline       486
Name: Event, dtype: int64

In [20]:
DataFrame.Subject_ID.value_counts()

966     171
984     121
1056    119
961     118
820     118
946     111
942     111
877     111
793     108
963     102
1058     99
962      99
937      74
Name: Subject_ID, dtype: int64

In [21]:
def Tonic_col(columns):
  tonic_col = []
  for x in columns:
    if 'tonic' in x:
      tonic_col.append(x)
  return tonic_col

In [22]:
from numpy import interp
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import roc_curve,auc, roc_auc_score, make_scorer
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import matplotlib.pylab as plt

In [25]:
rf = RandomForestClassifier(random_state = 42)
logreg = LogisticRegression(solver='lbfgs',penalty='l2',max_iter=500)
knn = KNeighborsClassifier()

In [23]:
def Training(empty_exp_list,Data,Phasic:bool,KFold:bool):
  
  if Phasic == True:
    col_list = Data.columns
    tonic_col = Tonic_col(col_list)
    DataFrame = Data.drop(tonic_col,axis =1)
  else:
    DataFrame = Data

  VRBaseline_Speech = DataFrame.loc[DataFrame['Event'].isin(['VR Baseline','Speech Emotion'])]
  VRBaseline_Speech["Event"] = VRBaseline_Speech["Event"].astype('category').cat.codes
  VRBaseline_FoodSelec = DataFrame.loc[DataFrame['Event'].isin(['VR Baseline','Food Selection'])]
  VRBaseline_FoodSelec["Event"] = VRBaseline_FoodSelec["Event"].astype('category').cat.codes
  Speech_FoodSelec = DataFrame.loc[DataFrame['Event'].isin(['Food Selection','Speech Emotion'])]
  Speech_FoodSelec["Event"] = Speech_FoodSelec["Event"].astype('category').cat.codes
    
  dict_exp = {'VR Baseline vs Speech':VRBaseline_Speech,
            'VR Baseline vs VR FoodSelec':VRBaseline_FoodSelec,
            'Speech vs VR FoodSelec':Speech_FoodSelec}

  for experiment, data in dict_exp.items():
    
    result_dict = {'experiment':experiment}
    
    if KFold == True:
      data = data.drop(['Subject_ID'], axis=1)

      feature = np.array(data.iloc[:,:-1])
      target = np.array(data.iloc[:,-1]) 

      cv = StratifiedKFold(n_splits=13)

      accuracy_rf=[]
      accuracy_logreg=[]
      accuracy_knn=[]
      auc_rf=[]
      auc_logreg=[]
      auc_knn=[]
      
      for train, test in cv.split(feature,target):
        subject_ID = None
        rf_cv=rf.fit(feature[train],target[train])
        logreg_cv=logreg.fit(feature[train],target[train])
        knn_cv=knn.fit(feature[train],target[train])

        y_pred_rf = rf_cv.predict(feature[test])
        fpr, tpr, threshhold = roc_curve(target[test],y_pred_rf)
        auc_RF = auc(fpr,tpr)
        accuracy_RF = accuracy_score(target[test],y_pred_rf)

        accuracy_rf.append(accuracy_RF)
        auc_rf.append(auc_RF)

        y_pred_logreg = logreg_cv.predict(feature[test])
        fpr, tpr, threshhold = roc_curve(target[test],y_pred_logreg)
        auc_LR = auc(fpr,tpr)
        accuracy_LR = accuracy_score(target[test],y_pred_logreg)

        accuracy_logreg.append(accuracy_LR)
        auc_logreg.append(auc_LR)

        y_pred_knn = knn_cv.predict(feature[test])
        fpr, tpr, threshhold = roc_curve(target[test],y_pred_logreg)
        auc_KNN = auc(fpr,tpr)
        accuracy_KNN = accuracy_score(target[test],y_pred_knn)

        accuracy_knn.append(accuracy_KNN)
        auc_knn.append(auc_KNN)

    else:
      subject_id = data['Subject_ID']
      data = data.drop(['Subject_ID'], axis=1)

      feature = np.array(data.iloc[:,:-1])
      target = np.array(data.iloc[:,-1])
      groups = np.array(subject_id)
    
      logo = LeaveOneGroupOut()
      logo.get_n_splits(feature, target, groups)
      logo.get_n_splits(groups=groups)

      accuracy_rf=[]
      accuracy_logreg=[]
      accuracy_knn=[]
      auc_rf=[]
      auc_logreg=[]
      auc_knn=[]

      subject_ID_avg = {}

      for train,test in logo.split(feature,target,groups):
        
        rf_cv=rf.fit(feature[train],target[train])
        logreg_cv=logreg.fit(feature[train],target[train])
        knn_cv=knn.fit(feature[train],target[train])

        y_pred_rf = rf_cv.predict(feature[test])
        fpr, tpr, threshhold = roc_curve(target[test],y_pred_rf)
        auc_RF = auc(fpr,tpr)
        accuracy_RF = accuracy_score(target[test],y_pred_rf)

        accuracy_rf.append(accuracy_RF)
        auc_rf.append(auc_RF)

        y_pred_logreg = logreg_cv.predict(feature[test])
        fpr, tpr, threshhold = roc_curve(target[test],y_pred_logreg)
        auc_LR = auc(fpr,tpr)
        accuracy_LR = accuracy_score(target[test],y_pred_logreg)

        accuracy_logreg.append(accuracy_LR)
        auc_logreg.append(auc_LR)

        y_pred_knn = knn_cv.predict(feature[test])
        fpr, tpr, threshhold = roc_curve(target[test],y_pred_knn)
        auc_KNN = auc(fpr,tpr)
        accuracy_KNN = accuracy_score(target[test],y_pred_knn)
        
        accuracy_knn.append(accuracy_KNN)
        auc_knn.append(auc_KNN)

        subject_ID = groups[test][0]
        subject_ID_avg[str(subject_ID)] = {'accuracy_rf':accuracy_RF,
                                          'auc_rf':auc_RF, 
                                          'accuracy_logreg':accuracy_LR,
                                          'auc_logreg':auc_LR,
                                          'accuracy_knn':accuracy_KNN,
                                          'auc_knn':auc_KNN}
    
    if KFold == True:
      pass
    else:
      result_dict['Subject_ID'] = subject_ID_avg

    result_dict['accuracy_score_testing'] = {'rf_cv':np.mean(accuracy_rf),
                                              'logreg_cv':np.mean(accuracy_logreg),
                                              'knn_cv':np.mean(accuracy_knn)}

    result_dict['auc_score_testing'] = {'rf_cv':np.mean(auc_rf),
                                              'logreg_cv':np.mean(auc_logreg),
                                              'knn_cv':np.mean(auc_knn)}

    empty_exp_list.append(result_dict)

  return empty_exp_list

In [26]:
KFoldCV_results = []
KFoldCV_results = Training(KFoldCV_results,DataFrame,Phasic = False, KFold = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [27]:
KFoldCV_results

[{'accuracy_score_testing': {'knn_cv': 0.8497020097020098,
   'logreg_cv': 0.814081774081774,
   'rf_cv': 0.8897851697851699},
  'auc_score_testing': {'knn_cv': 0.8152150125834337,
   'logreg_cv': 0.8152150125834337,
   'rf_cv': 0.8903873509136667},
  'experiment': 'VR Baseline vs Speech'},
 {'accuracy_score_testing': {'knn_cv': 0.7158974358974357,
   'logreg_cv': 0.7046153846153845,
   'rf_cv': 0.8215384615384613},
  'auc_score_testing': {'knn_cv': 0.7061768245978771,
   'logreg_cv': 0.7061768245978771,
   'rf_cv': 0.8222179669548091},
  'experiment': 'VR Baseline vs VR FoodSelec'},
 {'accuracy_score_testing': {'knn_cv': 0.4979757085020244,
   'logreg_cv': 0.5615384615384615,
   'rf_cv': 0.48576248313090414},
  'auc_score_testing': {'knn_cv': 0.5599354415143889,
   'logreg_cv': 0.5599354415143889,
   'rf_cv': 0.4844621949885108},
  'experiment': 'Speech vs VR FoodSelec'}]

In [28]:
KFoldCV = pd.DataFrame()
for index in KFoldCV_results:
  df = pd.DataFrame(index)
  KFoldCV = KFoldCV.append(df)

In [29]:
KFoldCV

,experiment,accuracy_score_testing,auc_score_testing
rf_cv,VR Baseline vs Speech,0.889785,0.890387
logreg_cv,VR Baseline vs Speech,0.814082,0.815215
knn_cv,VR Baseline vs Speech,0.849702,0.815215
rf_cv,VR Baseline vs VR FoodSelec,0.821538,0.822218
logreg_cv,VR Baseline vs VR FoodSelec,0.704615,0.706177
knn_cv,VR Baseline vs VR FoodSelec,0.715897,0.706177
rf_cv,Speech vs VR FoodSelec,0.485762,0.484462
logreg_cv,Speech vs VR FoodSelec,0.561538,0.559935
knn_cv,Speech vs VR FoodSelec,0.497976,0.559935


In [30]:
KFoldCV.to_csv("KFold_Phasic&Tonic.csv", index = True)

In [31]:
KFoldCV_Phasic_results = []
KFoldCV_Phasic_results = Training(KFoldCV_Phasic_results,DataFrame,Phasic = True, KFold = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
KFoldCV_Phasic_results

[{'accuracy_score_testing': {'knn_cv': 0.6177408177408176,
   'logreg_cv': 0.6548163548163548,
   'rf_cv': 0.6784476784476786},
  'auc_score_testing': {'knn_cv': 0.6544479702374439,
   'logreg_cv': 0.6544479702374439,
   'rf_cv': 0.6784385600175076},
  'experiment': 'VR Baseline vs Speech'},
 {'accuracy_score_testing': {'knn_cv': 0.6112820512820514,
   'logreg_cv': 0.6451282051282051,
   'rf_cv': 0.643076923076923},
  'auc_score_testing': {'knn_cv': 0.6455848561111719,
   'logreg_cv': 0.6455848561111719,
   'rf_cv': 0.642493708283182},
  'experiment': 'VR Baseline vs VR FoodSelec'},
 {'accuracy_score_testing': {'knn_cv': 0.5052901484480432,
   'logreg_cv': 0.4939271255060728,
   'rf_cv': 0.4918353576248313},
  'auc_score_testing': {'knn_cv': 0.4937629937629938,
   'logreg_cv': 0.4937629937629938,
   'rf_cv': 0.49160192581245216},
  'experiment': 'Speech vs VR FoodSelec'}]

In [32]:
KFoldCV_Phasic = pd.DataFrame()
for index in KFoldCV_Phasic_results:
  df = pd.DataFrame(index)
  KFoldCV_Phasic = KFoldCV_Phasic.append(df)

In [33]:
KFoldCV_Phasic.to_csv("KFold_Phasic.csv", index = True)

In [34]:
LOGO_results = []
LOGO_results = Training(LOGO_results,DataFrame,Phasic = False, KFold = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [35]:
LOGO_results

[{'Subject_ID': {'1056': {'accuracy_knn': 0.972972972972973,
    'accuracy_logreg': 1.0,
    'accuracy_rf': 0.918918918918919,
    'auc_knn': 0.972972972972973,
    'auc_logreg': 1.0,
    'auc_rf': 0.9189189189189189},
   '1058': {'accuracy_knn': 1.0,
    'accuracy_logreg': 0.958904109589041,
    'accuracy_rf': 1.0,
    'auc_knn': 1.0,
    'auc_logreg': 0.9594594594594594,
    'auc_rf': 1.0},
   '793': {'accuracy_knn': 0.7972972972972973,
    'accuracy_logreg': 0.7162162162162162,
    'accuracy_rf': 0.7567567567567568,
    'auc_knn': 0.7972972972972974,
    'auc_logreg': 0.7162162162162162,
    'auc_rf': 0.7567567567567568},
   '820': {'accuracy_knn': 0.7083333333333334,
    'accuracy_logreg': 0.8194444444444444,
    'accuracy_rf': 0.5833333333333334,
    'auc_knn': 0.7023166023166023,
    'auc_logreg': 0.8243243243243243,
    'auc_rf': 0.5714285714285714},
   '877': {'accuracy_knn': 0.9733333333333334,
    'accuracy_logreg': 0.9333333333333333,
    'accuracy_rf': 0.96,
    'auc_knn': 

In [36]:
LOGO = pd.DataFrame()
for index in LOGO_results:
  df = pd.DataFrame(index)
  LOGO = LOGO.append(df)

In [37]:
list_index = LOGO.index
indices = []
for index in range(0,len(list_index)):
  try:
    element = ast.literal_eval(list_index[index])
  except ValueError:
    indices.append(list_index[index])

In [38]:
indices

['rf_cv',
 'logreg_cv',
 'knn_cv',
 'rf_cv',
 'logreg_cv',
 'knn_cv',
 'rf_cv',
 'logreg_cv',
 'knn_cv']

In [39]:
Mean_LOGO = pd.DataFrame(LOGO.loc[indices])
Mean_LOGO = Mean_LOGO.drop_duplicates()
Mean_LOGO = Mean_LOGO.drop(columns = ['Subject_ID'])
Mean_LOGO.to_csv("MeanLOGO_Phasic&Tonic.csv", index = True)

In [40]:
Subject_LOGO = LOGO.drop(indices).drop(columns = ['accuracy_score_testing','auc_score_testing'])

In [43]:
list_=[]
for w in range(0,len(Subject_LOGO)):
  index = Subject_LOGO.index
  list_.append(index[w])


In [44]:
dummy_df = pd.DataFrame()
for one in range(0,len(Subject_LOGO)):
  Dict = Subject_LOGO['Subject_ID'][one]
  df = pd.DataFrame(Dict,index=[list_[one]])
  exp = Subject_LOGO['experiment'][one]
  df['experiment'] = exp
  dummy_df = dummy_df.append(df)


In [46]:
dummy_df.to_csv("LOGO_Phasic&Tonic.csv", index = True)

In [47]:
LOGO_phasic_results = []
LOGO_phasic_results = Training(LOGO_phasic_results,DataFrame,Phasic = True, KFold = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [48]:
LOGO_phasic_results

[{'Subject_ID': {'1056': {'accuracy_knn': 0.7162162162162162,
    'accuracy_logreg': 0.7567567567567568,
    'accuracy_rf': 0.7702702702702703,
    'auc_knn': 0.716216216216216,
    'auc_logreg': 0.7567567567567567,
    'auc_rf': 0.7702702702702703},
   '1058': {'accuracy_knn': 0.5616438356164384,
    'accuracy_logreg': 0.5753424657534246,
    'accuracy_rf': 0.589041095890411,
    'auc_knn': 0.56006006006006,
    'auc_logreg': 0.5731981981981982,
    'auc_rf': 0.5897147147147147},
   '793': {'accuracy_knn': 0.581081081081081,
    'accuracy_logreg': 0.6081081081081081,
    'accuracy_rf': 0.7162162162162162,
    'auc_knn': 0.581081081081081,
    'auc_logreg': 0.6081081081081081,
    'auc_rf': 0.7162162162162162},
   '820': {'accuracy_knn': 0.6527777777777778,
    'accuracy_logreg': 0.7222222222222222,
    'accuracy_rf': 0.8194444444444444,
    'auc_knn': 0.6482625482625484,
    'auc_logreg': 0.7166023166023165,
    'auc_rf': 0.818918918918919},
   '877': {'accuracy_knn': 0.57333333333333

In [49]:
LOGO_phasic = pd.DataFrame()
for index in LOGO_phasic_results:
  df = pd.DataFrame(index)
  LOGO_phasic = LOGO_phasic.append(df)

In [50]:
list_index = LOGO_phasic.index
indices = []
for index in range(0,len(list_index)):
  try:
    element = ast.literal_eval(list_index[index])
  except ValueError:
    indices.append(list_index[index])

In [51]:
Mean_LOGO_phasic = pd.DataFrame(LOGO_phasic.loc[indices])
Mean_LOGO_phasic = Mean_LOGO_phasic.drop_duplicates()
Mean_LOGO_phasic = Mean_LOGO_phasic.drop(columns = ['Subject_ID'])
Mean_LOGO_phasic.to_csv("MeanLOGO_Phasic.csv", index = False)

In [52]:
Subject_LOGO = LOGO_phasic.drop(indices).drop(columns = ['accuracy_score_testing','auc_score_testing'])

In [53]:
list_=[]
for w in range(0,len(Subject_LOGO)):
  index = Subject_LOGO.index
  list_.append(index[w])

In [54]:
dummy_df = pd.DataFrame()
for one in range(0,len(Subject_LOGO)):
  Dict = Subject_LOGO['Subject_ID'][one]
  df = pd.DataFrame(Dict,index=[list_[one]])
  exp = Subject_LOGO['experiment'][one]
  df['experiment'] = exp
  dummy_df = dummy_df.append(df)

In [56]:
dummy_df.to_csv("LOGO_Phasic.csv", index = True)